In [4]:
import torch
from Noise import NoiseScheduler
from model import U_net
from tqdm import tqdm
import torchvision
import os

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_size = 28

num_timesteps = 1000
noise_scheduler = NoiseScheduler(num_timesteps=num_timesteps, beta_start=0.0001, beta_end=0.02, device=device)

model = U_net(device)
model.load_state_dict(torch.load(os.path.join("Save_model", f"model_DDPM.pth"), map_location=device))
model.to(device)

U_net(
  (time_mlp): TimeEmbedding(
    (lin1): Linear(in_features=256, out_features=1024, bias=True)
    (Silu): SiLU()
    (lin2): Linear(in_features=1024, out_features=256, bias=True)
  )
  (Down_blocks): ModuleList(
    (0): Down_block(
      (Res_block): ResBlock(
        (conv1): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): SiLU()
        )
        (time_mlp): Sequential(
          (0): SiLU()
          (1): Linear(in_features=256, out_features=64, bias=True)
        )
        (conv2): Sequential(
          (0): SiLU()
          (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (shortcut): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Down_block(
      (Res_block): ResBlock(
        (conv1): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1),

In [6]:
#sample
xt = torch.randn(1, 1, image_size, image_size).to(device)

for i in tqdm(reversed(range(num_timesteps))):
    noise_pred = model(xt, torch.as_tensor(i).unsqueeze(0).to(device))

    xt, _ = noise_scheduler.sample_prev_timestep(xt, noise_pred, torch.as_tensor(i).to(device))

    # denormalizace z [-1, 1] → [0, 1]
    img = torch.clamp(xt, -1., 1.).detach().cpu()
    img = img * 0.5 + 0.5

    # tensor -> PIL image
    to_pil = torchvision.transforms.ToPILImage()
    img_pil = to_pil(img[0]) 

    # cesta + přípona
    save_path = os.path.join(os.getcwd(), "generated_imgs", f"x_{i}.png")

    # uložení
    img_pil.save(save_path)
    img_pil.close()

1000it [00:08, 116.05it/s]
